In [64]:
# %load_ext autoreload
%autoreload 2
import torch
from datasets import TripletAudio

K, MAX_CLOSE_NEG, MAX_FAR_NEG = 5, 15, 15
BATCH_SIZE = 1

triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [98]:
%autoreload 2
# Set up the network and training parameters
from networks import AnchorNet, EmbeddingNet, TripletNet
from losses import TripletLoss
import torch.optim as optim
from torch.optim import lr_scheduler

INPUT_D, OUTPUT_D = 192, 128
MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT = 0.5, 1e-3, 5, 1000

#define model
anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), INPUT_D, OUTPUT_D)
embedding_net = EmbeddingNet(anchor_net)
model = TripletNet(embedding_net)

loss_fn = TripletLoss(MARGIN)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [99]:
%autoreload 2
from trainer import fit
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT)

Train: [0/48048 (0%)]	Loss: 0.000000
Train: [1000/48048 (2%)]	Loss: 48.292651
Train: [2000/48048 (4%)]	Loss: 56.032098
Train: [3000/48048 (6%)]	Loss: 37.911216
Train: [4000/48048 (8%)]	Loss: 42.142341
Train: [5000/48048 (10%)]	Loss: 44.214018
Train: [6000/48048 (12%)]	Loss: 55.352000
Train: [7000/48048 (15%)]	Loss: 33.549665
Train: [8000/48048 (17%)]	Loss: 43.992335
Train: [9000/48048 (19%)]	Loss: 34.851070
Train: [10000/48048 (21%)]	Loss: 31.263697
Train: [11000/48048 (23%)]	Loss: 30.159823
Train: [12000/48048 (25%)]	Loss: 26.266181
Train: [13000/48048 (27%)]	Loss: 31.043728
Train: [14000/48048 (29%)]	Loss: 28.150132
Train: [15000/48048 (31%)]	Loss: 23.868521
Train: [16000/48048 (33%)]	Loss: 24.580456
Train: [17000/48048 (35%)]	Loss: 23.984632
Train: [18000/48048 (37%)]	Loss: 23.977490
Train: [19000/48048 (40%)]	Loss: 19.264973
Train: [20000/48048 (42%)]	Loss: 23.102680
Train: [21000/48048 (44%)]	Loss: 19.204739
Train: [22000/48048 (46%)]	Loss: 21.792413
Train: [23000/48048 (48%)]	Los

Train: [42000/48048 (87%)]	Loss: 1.981519
Train: [43000/48048 (89%)]	Loss: 3.274859
Train: [44000/48048 (92%)]	Loss: 2.773484
Train: [45000/48048 (94%)]	Loss: 2.348598
Train: [46000/48048 (96%)]	Loss: 3.301717
Train: [47000/48048 (98%)]	Loss: 2.298358
Train: [48000/48048 (100%)]	Loss: 3.227425
Epoch: 4/5. Train set: Average loss: 2.8888
Epoch: 4/5. Validation set: Average loss: 2.7803
Train: [0/48048 (0%)]	Loss: 47.872681
Train: [1000/48048 (2%)]	Loss: 2.191969
Train: [2000/48048 (4%)]	Loss: 2.290140
Train: [3000/48048 (6%)]	Loss: 2.457664
Train: [4000/48048 (8%)]	Loss: 2.176928
Train: [5000/48048 (10%)]	Loss: 3.086573
Train: [6000/48048 (12%)]	Loss: 2.756470
Train: [7000/48048 (15%)]	Loss: 2.286622
Train: [8000/48048 (17%)]	Loss: 2.259851
Train: [9000/48048 (19%)]	Loss: 1.767354
Train: [10000/48048 (21%)]	Loss: 2.574614
Train: [11000/48048 (23%)]	Loss: 2.280500
Train: [12000/48048 (25%)]	Loss: 2.667292
Train: [13000/48048 (27%)]	Loss: 2.186673
Train: [14000/48048 (29%)]	Loss: 2.441596